In [2]:
import findspark
findspark.init("/opt/spark")

In [3]:
from pyspark.sql import SparkSession, functions as F
from pyspark import SparkContext
import configparser

In [4]:
config = configparser.RawConfigParser()

config.read('/dataops/db_conn')
accessKeyId = config.get('DB', 'user_name')
secretAccessKey = config.get('DB', 'password')

In [5]:
spark = SparkSession.builder \
.appName("final project silver layer") \
.master("local[2]") \
.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0,io.delta:delta-core_2.12:2.4.0,io.delta:delta-storage:2.4.0") \
.config("spark.hadoop.fs.s3a.access.key", accessKeyId) \
.config("spark.hadoop.fs.s3a.secret.key", secretAccessKey) \
.config("spark.hadoop.fs.s3a.path.style.access", True) \
.config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
.config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
.getOrCreate() 

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-core_2.12 added as a dependency
io.delta#delta-storage added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4c122f7e-894a-42a3-914f-949934733b41;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 393ms :: artifacts dl 17ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from central in [default]
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.0 from central in [d

## **read from minio in the form deltatable**

In [8]:
from delta.tables import *

In [6]:
deltaPath = f"s3a://tmdb-silver/cast"

In [11]:
#DeltaTable ile forPath metoduna spark session'ı ve 
#delta path bilgisi vererek delta table formatında okuyoruz.
cast_delta = DeltaTable.forPath(spark, deltaPath)

In [12]:
type(cast_delta)

delta.tables.DeltaTable

In [13]:
cast_delta.toDF().show()

23/11/18 02:56:38 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+-----+-------+------------------+--------------------+------+-------+-------------------+
|movie_id|title|cast_id|         character|           credit_id|gender|     id|               name|
+--------+-----+-------+------------------+--------------------+------+-------+-------------------+
|   73935|Vamps|      3|             Goody|52fe48b3c3a368484...|     0|   5588| Alicia Silverstone|
|   73935|Vamps|      4|             Stacy|52fe48b3c3a368484...|     1|  78080|     Krysten Ritter|
|   73935|Vamps|      5|          Cisserus|52fe48b3c3a368484...|     1|  10205|   Sigourney Weaver|
|   73935|Vamps|      6|             Danny|52fe48b3c3a368484...|     2|   6105|      Richard Lewis|
|   73935|Vamps|      7|   Dr. Van Helsing|52fe48b3c3a368484...|     2|  12900|      Wallace Shawn|
|   73935|Vamps|      8|              Vlad|52fe48b3c3a368484...|     2|  56890|   Malcolm McDowell|
|   73935|Vamps|      9|          Renfield|52fe48b3c3a368484...|     2|   6212|           Zak Orth|


In [15]:
cast_delta.toDF().printSchema()

root
 |-- movie_id: long (nullable = true)
 |-- title: string (nullable = true)
 |-- cast_id: long (nullable = true)
 |-- character: string (nullable = true)
 |-- credit_id: string (nullable = true)
 |-- gender: long (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)

